In [14]:
!pip install langchain
!pip install python-dotenv
!pip install tavily-python

In [15]:
!pip install google-generativeai

In [16]:
!pip show google-generativeai
!pip show tavily-python
!pip show langchain

Name: google-generativeai
Version: 0.8.3
Summary: Google Generative AI High level API client library and tools.
Home-page: https://github.com/google/generative-ai-python
Author: Google LLC
Author-email: googleapis-packages@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: google-ai-generativelanguage, google-api-core, google-api-python-client, google-auth, protobuf, pydantic, tqdm, typing-extensions
Required-by: 
Name: tavily-python
Version: 0.5.0
Summary: Python wrapper for the Tavily API
Home-page: https://github.com/tavily-ai/tavily-python
Author: Tavily AI
Author-email: support@tavily.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: httpx, requests, tiktoken
Required-by: 
Name: langchain
Version: 0.3.7
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aioht

In [27]:
from tavily import TavilyClient
import google.generativeai as genai
from langchain.memory import ConversationBufferMemory
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')

# Initialize Tavily client
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

# Initialize Google GenAI client
genai.configure(api_key=GOOGLE_API_KEY)

# Initialize conversation memory
memory = ConversationBufferMemory()


In [28]:
import google.generativeai as genai

# Tavily search function
def search_tavily(query):
    try:
        response = tavily_client.search(query)
        results = response.get("results", [])
        return results
    except Exception as e:
        return f"Error with Tavily: {e}"

# Google GenAI query function
def query_google_genai(prompt):
    try:
        model = genai.GenerativeModel('models/gemini-1.5-flash')
        response = model.generate_content(prompt)
        # text_response = response.candidates[0].content.parts[0].text.strip()
        text_response = response.text if response.text else "No response from Google GenAI."
    except Exception as e:
        return f"Error with Google GenAI: {e}"
    return text_response

# Chatbot function
def chatbot_response(user_input):
    # Search Tavily
    tavily_results = search_tavily(user_input)
    if isinstance(tavily_results, list):
        search_output = "\n".join(
            [f"{result['title']}: {result['url']}" for result in tavily_results]
        )
    else:
        search_output = f"Error: {tavily_results}"

    # Generate a response using Google GenAI
    genai_response = query_google_genai(user_input)

    # Combine Tavily and GenAI responses
    full_response = (
        f"**Tavily Search Results:**\n{search_output}\n\n"
        f"**Google GenAI Response:**\n{genai_response}"
    )

    # Save conversation to memory
    memory.save_context({"input": user_input}, {"output": full_response})

    return full_response

In [29]:
print("Chatbot is ready! Type 'exit' to quit.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Goodbye!")
        break

    response = chatbot_response(user_input)
    print(f"Bot: {response}")


Chatbot is ready! Type 'exit' to quit.

You: russia capital city
Bot: **Tavily Search Results:**
Moscow - Simple English Wikipedia, the free encyclopedia: https://simple.wikipedia.org/wiki/Moscow
History Milestones / History Milestones / Moscow City Web Site: https://www.mos.ru/en/city/about/
Moscow | History, Geography, Population, & Map | Britannica: https://www.britannica.com/place/Moscow
6 Russian capitals: How the country's heart changed ... - Russia Beyond: https://www.rbth.com/history/326278-6-russian-capitals
Moscow - New World Encyclopedia: https://www.newworldencyclopedia.org/entry/Moscow

**Google GenAI Response:**
Moscow

You: exit
Goodbye!
